In [ ]:
import os.path as osp

import torch
import torch.nn as nn

from abl.bridge import SimpleBridge
from abl.evaluation import SemanticsMetric, SymbolMetric
from abl.learning import ABLModel, BasicNN
from abl.reasoning import KBBase, Reasoner
from abl.utils import ABLLogger, print_log
from examples.mnist_add.datasets.get_mnist_add import get_mnist_add
from examples.models.nn import LeNet5

In [ ]:
# Build logger
print_log("Abductive Learning on the MNIST Add example.", logger="current")

# Retrieve the directory of the Log file and define the directory for saving the model weights.
log_dir = ABLLogger.get_current_instance().log_dir
weights_dir = osp.join(log_dir, "weights")

### Machine Learning Part

In [ ]:
# Build necessary components for BasicNN
cls = LeNet5(num_classes=10)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cls.parameters(), lr=0.001, betas=(0.9, 0.99))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Build BasicNN
# The function of BasicNN is to wrap NN models into the form of an sklearn estimator
base_model = BasicNN(
    cls,
    criterion,
    optimizer,
    device,
    batch_size=32,
    num_epochs=1,
)

In [ ]:
# Build ABLModel
# The main function of the ABL model is to serialize data and
# provide a unified interface for different machine learning models
model = ABLModel(base_model)

### Logic Part

In [ ]:
# Build knowledge base and reasoner
class AddKB(KBBase):
    def __init__(self, pseudo_label_list):
        super().__init__(pseudo_label_list)

    # Implement the deduction function
    def logic_forward(self, nums):
        return sum(nums)


kb = AddKB(pseudo_label_list=list(range(10)))
reasoner = Reasoner(kb, dist_func="confidence")

### Datasets and Evaluation Metrics

In [ ]:
# Get training and testing data
train_data = get_mnist_add(train=True, get_pseudo_label=True)
test_data = get_mnist_add(train=False, get_pseudo_label=True)

In [ ]:
# Set up metrics
metric_list = [SymbolMetric(prefix="mnist_add"), SemanticsMetric(kb=kb, prefix="mnist_add")]

### Bridge Machine Learning and Logic Reasoning

In [ ]:
bridge = SimpleBridge(model, reasoner, metric_list)

### Train and Test

In [ ]:
bridge.train(train_data, loops=5, segment_size=10000, save_interval=1, save_dir=weights_dir)
bridge.test(test_data)